# 01. Basic about Tensorflow and Tensorboard
- Good resources to read about Tensorboard: [here](https://www.tensorflow.org/guide/summaries_and_tensorboard) from TF website, and [here](https://www.datacamp.com/community/tutorials/tensorboard-tutorial).

## Quick summary about Tensorboard:
1. tf.summary: create the TensorFlow graph that you'd like to collect summary data from, and decide which nodes you would like to annotate. This means that whatever value you'd like to be displayed, you should encapsulate as a tf.summary object.
2. tf.summary.scalar: e.g., record how the learning rate varies over time, and how the objective function is changing.
3. tf.summary.histogram: e.g.,  visualize the distributions of activations coming off a particular layer, or the distribution of gradients or weights.
4. tf.summary.merge_all: to combine all summaries into a single op that generates all the summary data.
5. tf.summary.FileWriter: write this summary data to disk.

In [1]:
import tensorflow as tf
import numpy as np

/home/sonvx/anaconda2/envs/ipykernel_py3_ml/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# Showing graph on Jupyter Notebook
# source: https://stackoverflow.com/questions/38189119/simple-way-to-visualize-a-tensorflow-graph-in-jupyter
from IPython.display import clear_output, Image, display, HTML

def strip_consts(graph_def, max_const_size=32):
    """Strip large constant values from graph_def."""
    strip_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = strip_def.node.add() 
        n.MergeFrom(n0)
        if n.op == 'Const':
            tensor = n.attr['value'].tensor
            size = len(tensor.tensor_content)
            if size > max_const_size:
                tensor.tensor_content = "<stripped %d bytes>"%size
    return strip_def

def show_graph(graph_def, max_const_size=32):
    """Visualize TensorFlow graph."""
    if hasattr(graph_def, 'as_graph_def'):
        graph_def = graph_def.as_graph_def()
    strip_def = strip_consts(graph_def, max_const_size=max_const_size)
    code = """
        <script>
          function load() {{
            document.getElementById("{id}").pbtxt = {data};
          }}
        </script>
        <link rel="import" href="https://tensorboard.appspot.com/tf-graph-basic.build.html" onload=load()>
        <div style="height:600px">
          <tf-graph-basic id="{id}"></tf-graph-basic>
        </div>
    """.format(data=repr(str(strip_def)), id='graph'+str(np.random.rand()))

    iframe = """
        <iframe seamless style="width:1200px;height:620px;border:0" srcdoc="{}"></iframe>
    """.format(code.replace('"', '&quot;'))
    display(HTML(iframe))

## 1. Play with operations and sessions

In [3]:
logs_path_1 = '../../../my_data/tf_logs_tb'

In [4]:
tf.reset_default_graph()
x = 2
y = 3
op1 = tf.add(x, y)
op2 = tf.multiply(x,y)
op3 = tf.pow(op1, op2)

In [5]:
with tf.Session() as sess:
    op3 = sess.run(op3)
    writer = tf.summary.FileWriter(logs_path_1, sess.graph)

In [6]:
# Run tensorboard to see:
# !tensorboard --logdir=$logs_path --port 8890
# OR:
show_graph(tf.get_default_graph().as_graph_def())

## 2. Play with operations and placeholders

In [7]:
# A bit more complicate one:
tf.reset_default_graph()
x_arr = np.array([1,2,3,4])
y_arr = np.array([3,4,5,6])

In [8]:
x_ph = tf.placeholder(tf.int32, None)
y_ph = tf.placeholder(tf.int32, None)

In [9]:
op1 = tf.add(x_ph, y_ph)
op2 = tf.multiply(x_ph,y_ph)
op3 = tf.pow(op1, op2)

In [10]:
with tf.Session() as sess:
    out_op1 = sess.run(op1, feed_dict={x_ph: x_arr, y_ph:y_arr})
    print("op1 (x+y) = ", out_op1)
    out_op2 = sess.run(op2, feed_dict={x_ph: x_arr, y_ph:y_arr})
    print("op2 (x*y)= ", out_op2)
    out_op3 = sess.run(op3, feed_dict={x_ph: x_arr, y_ph:y_arr})
    print("op3 (x*y)= ", out_op3)
    
    # writer.add_graph(sess.graph)
    writer = tf.summary.FileWriter(logs_path_1, sess.graph)

op1 (x+y) =  [ 4  6  8 10]
op2 (x*y)=  [ 3  8 15 24]
op3 (x*y)=  [         64     1679616           0 -1593835520]


In [11]:
# Check tensorboard
# !tensorboard --logdir=$logs_path_1 --port 8890
show_graph(tf.get_default_graph().as_graph_def())

## 3. Adding Tensorboard

In [12]:
logs_path_2 = '../../../my_data/tf_logs_scalars'

In [13]:
x_2d_arr = np.array([[1,2,3,4],
                  [4,3,2,1]
                 ])
y_2d_arr = np.array([[3,4,5,6],
                  [6,5,4,3]
                 ])

In [23]:
tf.reset_default_graph()
x_ph = tf.placeholder(tf.int32, None)
y_ph = tf.placeholder(tf.int32, None)
op1 = tf.add(x_ph, y_ph)
op2 = tf.multiply(x_ph,y_ph)
op3 = tf.pow(op1, op2)

In [24]:
# Name scope allows you to group various summaries together
# Summaries having the same name_scope will be displayed on the same row on the Tensorboard
with tf.name_scope('oper_summaries'):
    # Summaries need to display on the Tensorboard
    # Whenever need to record the op1, feed the op1 to this placeholder
    tf_op1_ph = tf.placeholder(tf.int32, shape=[4], name='op1') 
    # Create a scalar summary object for the loss so Tensorboard knows how to display it
    # tf_op1_summary = tf.summary.scalar('op1', tf.reduce_mean(tf_op1_ph))
    tf_op1_summary = tf.summary.histogram('op1', tf_op1_ph)
    
    # Whenever need to record the op1, feed the op1 to this placeholder
    tf_op2_ph = tf.placeholder(tf.int32, shape=None, name='op2') 
    # Create a scalar summary object for the loss so Tensorboard knows how to display it
    # tf_op2_summary = tf.summary.scalar('op2', tf.reduce_mean(tf_op2_ph))
    tf_op2_summary = tf.summary.histogram('op2', tf_op2_ph)

performance_summaries = tf.summary.merge([tf_op1_summary,tf_op2_summary])

In [25]:
with tf.Session() as sess:
    summ_writer = tf.summary.FileWriter(logs_path_2, sess.graph)
    
    idx = 0
    for x_1d_arr, y_1d_arr in zip(x_2d_arr, y_2d_arr):
        out_op1 = sess.run(op1, feed_dict={x_ph: x_1d_arr, y_ph:y_1d_arr})
        print(type(out_op1), out_op1.shape)
        print("op1 (x+y) = ", out_op1)
        out_op2 = sess.run(op2, feed_dict={x_ph: x_1d_arr, y_ph:y_1d_arr})
        print("op2 (x*y)= ", out_op2)
        print(type(out_op1), out_op2.shape)
        out_op3 = sess.run(op3, feed_dict={x_ph: x_1d_arr, y_ph:y_1d_arr})
        print("op3 (x*y)= ", out_op3)
        summ = sess.run(performance_summaries, feed_dict={tf_op1_ph: out_op1, tf_op2_ph: out_op2})
        summ_writer.add_summary(summ, idx)
        idx += 1
    # writer.add_graph(sess.graph)
    writer = tf.summary.FileWriter(logs_path_2, sess.graph)

<class 'numpy.ndarray'> (4,)
op1 (x+y) =  [ 4  6  8 10]
op2 (x*y)=  [ 3  8 15 24]
<class 'numpy.ndarray'> (4,)
op3 (x*y)=  [         64     1679616           0 -1593835520]
<class 'numpy.ndarray'> (4,)
op1 (x+y) =  [10  8  6  4]
op2 (x*y)=  [24 15  8  3]
<class 'numpy.ndarray'> (4,)
op3 (x*y)=  [-1593835520           0     1679616          64]


In [26]:
# !tensorboard --logdir=$logs_path_2 --port 8890
show_graph(tf.get_default_graph().as_graph_def())

# Conclusions: after this Notebook, you should know:
- How to use operations, sessions, placeholders in Tensorflow.
- How to use Tensorboard to visualize the computational graph.